## Chroma

### **Chroma란?**

Chroma는 AI 애플리케이션을 위한 **오픈 소스 벡터 데이터베이스**로, 의미론적 검색과 대규모 임베딩 데이터를 효율적으로 저장하고 검색하는 기능을 제공합니다.

### **특징**

- **AI 네이티브**: 머신러닝 및 AI 모델과의 통합이 용이함
- **오픈 소스**: Apache 2.0 라이선스 제공
- **고속 검색**: 벡터 임베딩 기반의 유사 검색 지원
- **쉬운 사용**: Python 및 LangChain과 쉽게 연동 가능

### **활용 사례**

- 문서 검색(RAG, 검색 기반 생성 AI)
- 추천 시스템
- 자연어 처리(NLP)

즉, Chroma는 **AI 애플리케이션을 위한 벡터 데이터 저장 및 검색을 간편하게 지원하는 데이터베이스**입니다.


In [3]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader  # 텍스트 파일 로더
from langchain_openai.embeddings import OpenAIEmbeddings  # OpenAI 임베딩 사용
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_chroma import Chroma  # 벡터 DB (Chroma) 사용

# 1. 환경 변수 로드 (.env 파일에서 API 키 불러오기)
load_dotenv()

# 2. 벡터 데이터베이스 저장 경로 설정
DB_PATH = "../../db/chroma_db"

# 3. 텍스트 파일을 로드하고 문서를 분할하는 함수 정의
def load_and_split_text(file_path, splitter):
    """
    주어진 텍스트 파일을 로드한 후, 설정된 Splitter를 사용하여 문서를 나누는 함수.
    
    Args:
        file_path (str): 로드할 파일 경로
        splitter (RecursiveCharacterTextSplitter): 텍스트 분할기 객체

    Returns:
        list: 분할된 문서 리스트
    """
    if not os.path.exists(file_path):
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return []
    
    try:
        loader = TextLoader(file_path)  # 텍스트 파일 로드
        return loader.load_and_split(splitter)  # 분할하여 반환
    except Exception as e:
        print(f"파일 로드 오류 ({file_path}): {e}")
        return []

# 4. 텍스트 분할기 설정 (600자 단위로 나누고, 100자 겹침 포함)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)

# 5. 두 개의 텍스트 파일 로드 및 분할
split_doc1 = load_and_split_text("../../data/ai-terminology.txt", text_splitter)
split_doc2 = load_and_split_text("../../data/finance-terminology.txt", text_splitter)

# 6. 문서 개수 출력
print(f"AI 문서 개수: {len(split_doc1)}")
print(f"금융 문서 개수: {len(split_doc2)}")

# 7. 모든 문서 합치기
all_documents = split_doc1 + split_doc2

# 8. Chroma 벡터 DB 생성 및 저장
try:
    persist_db = Chroma.from_documents(
        documents=all_documents,
        embedding=OpenAIEmbeddings(),  # OpenAI Embeddings 사용
        persist_directory=DB_PATH,  # 벡터 DB 저장 위치 지정
        collection_name="my_vector_db",  # 데이터베이스 컬렉션 이름
    )
    print("Chroma 데이터베이스 저장 완료!")
except Exception as e:
    print(f"Chroma 데이터베이스 저장 오류: {e}")

# 9. 저장된 데이터 확인
try:
    retrieved_docs = persist_db.get()  # Chroma DB에서 데이터 조회
    print(f"저장된 벡터 개수: {len(retrieved_docs['ids'])}")
except Exception as e:
    print(f"데이터 조회 오류: {e}")

# 10. 유사도 검색 함수 정의
def search_query(query, k=2):
    """
    사용자 입력(query)에 대해 가장 유사한 문서를 검색하는 함수.

    Args:
        query (str): 검색할 문장 (예: "Transformer 개념 설명")
        k (int, optional): 검색할 문서 개수. Defaults to 2.

    Returns:
        None: 검색 결과를 출력
    """
    try:
        results = persist_db.similarity_search(query, k=k)  # 유사도 검색 수행
        print(f"\n [Query]: {query}\n")
        for i, doc in enumerate(results):
            print(f" [Result {i+1}]: {doc.page_content[:300]}...\n")  # 검색 결과 출력
    except Exception as e:
        print(f"검색 오류: {e}")

# 11. 검색 테스트 실행
search_query("Transformer 에 대해 설명해줘", k=2)
search_query("TF-IDF 개념이 뭐야?", k=2)


파일 로드 오류 (../../data/ai-terminology.txt): Error loading ../../data/ai-terminology.txt
파일 로드 오류 (../../data/finance-terminology.txt): Error loading ../../data/finance-terminology.txt
AI 문서 개수: 0
금융 문서 개수: 0
Chroma 데이터베이스 저장 오류: Expected Embeddings to be non-empty list or numpy array, got [] in upsert.
데이터 조회 오류: name 'persist_db' is not defined
검색 오류: name 'persist_db' is not defined
검색 오류: name 'persist_db' is not defined
